# Node Classification in Knowledge Graphs - Cora Dataset

## Installation of Pytorch Geometric

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
!python -c "import torch; print(torch.version.cuda)"

In [ ]:
!pip3 install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip3 install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip3 install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip3 install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip3 install torch-geometric

## Import and visualize the dataset

In [5]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [6]:
dataset = Planetoid(root = 'data/Planetoid', name = 'Cora', transform = NormalizeFeatures())

Processing...
Done!


In [10]:
print(f"NUmber of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"NUmber of classes: {dataset.num_classes}")
print("---------------------------------------")

data = dataset[0]

print(data)
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Number of training nodes: {data.train_mask.sum()}")
print(f"Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}")
print(f"Is undirected: {data.is_undirected()}")

NUmber of graphs: 1
Number of features: 1433
NUmber of classes: 7
---------------------------------------
Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Number of nodes: 2708
Number of edges: 10556
Number of training nodes: 140
Training node label rate: 0.05
Is undirected: True


In [11]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [12]:
class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):

    super(GCN, self).__init__()
    torch.manual_seed(10)

    # Initialize GCN layers
    self.conv1 = GCNConv(dataset.num_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, hidden_channels)
    self.out = Linear(hidden_channels, dataset.num_classes)

  def forward(self, x, edge_index):

    # First message passing layer
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p = 0.2, training = self.training)

    # Second message passing layer
    x = self.conv2(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p = 0.2, training = self.training)


In [13]:
model = GCN(hidden_channels = 16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 16)
  (out): Linear(in_features=16, out_features=7, bias=True)
)
